In [ ]:
import pandas as pd
import numpy as np
import librosa
import h5py
import math
import warnings

In [73]:
one_to_loop_ratio = 1
loop_unit_multi_sample_pct = 0.2

In [92]:
meta_data = pd.read_csv('audio_metadata - filtered.csv', sep=',')
# meta_data.drop(columns=['group', 'category', 'sub_category'], inplace=True)

In [93]:
df_id = np.array(meta_data['id'].astype(int)).flatten()

wavs = []
srs = []

def load_sample(f, sample_id):
    group = f["audio_data"][str(sample_id)]
    waveform = np.array(group["waveform"])
    sample_rate = np.array(group["sample_rate"])
    return waveform, sample_rate


with h5py.File('filtered_audio_data.h5', "r") as f:
    for i in df_id:
        waveform, sr = load_sample(f, i)  # load sample with id=42
        wavs.append(waveform)
        srs.append(sr)


samples = pd.DataFrame({'id': df_id, 'waveform': wavs, 'sample_rate': srs, })

In [94]:
samples = samples.set_index('id').join(meta_data.set_index('id'))
one_shots = samples.copy()
loops = samples.copy()

In [96]:
samples

,waveform,sample_rate,file_path,file_name,group,category,sub_category,Reverse,Forward,One_Shot,...,Chord,Glitch,Vibraslap,Timpani,Vocal FX,China,Cowbell,Bell,Orchestra,Metalic Tap
id,,,,,,,,,,,,,,,,,,,,,
5487,"[5.164626e-05, -0.000103295286, 2.5824858e-05,...",44100,808 and kick/heavy kick.wav,Heavy Kick.wav,Drum,Kick,NaN,0,1,1,...,0,0,0,0,0,0,0,0,0,0
5087,"[-0.0004272461, -0.0016479492, -0.007232666, -...",44100,808 samples/808/bass (12).wav,bass (12).wav,Drum,Kick,NaN,0,1,1,...,0,0,0,0,0,0,0,0,0,0
5084,"[-0.0009765625, -0.0026550293, -0.009033203, -...",44100,808 samples/808/bass (13).wav,bass (13).wav,Drum,Kick,NaN,0,1,1,...,0,0,0,0,0,0,0,0,0,0
5083,"[-0.00039672852, -0.00045776367, -0.0014953613...",44100,808 samples/808/bass (14).wav,bass (14).wav,Drum,Kick,NaN,0,1,1,...,0,0,0,0,0,0,0,0,0,0
5098,"[-0.0029296875, -0.010894775, -0.051208496, -0...",44100,808 samples/808/bass (16).wav,bass (16).wav,Drum,Kick,NaN,0,1,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7061,"[0.0014566779, -0.0014469028, 0.009420276, 0.5...",44100,we unite - ultimate future bounce (sample pack...,16 Kick D#.wav,Drum,Kick,NaN,0,1,1,...,0,0,0,0,0,0,0,0,0,0
7059,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",44100,we unite - ultimate future bounce (sample pack...,17 Kick E.wav,Drum,Kick,NaN,0,1,1,...,0,0,0,0,0,0,0,0,0,0
7053,"[-0.018602252, 0.06855637, 0.044089437, 0.1702...",44100,we unite - ultimate future bounce (sample pack...,18 Kick A.wav,Drum,Kick,NaN,0,1,1,...,0,0,0,0,0,0,0,0,0,0


In [97]:
if 'one_shot_intent' and 'loop_id' in samples.columns:
    print("'loop_id' and 'one_shot_intent' are present")
    one_shots = one_shots[one_shots['One_Shot'] == 1 and one_shots['one_shot_intent']==1]
    loops = loops[loops['loop_id'] > 0]
else:
    print("'loop_id' and 'one_shot_intent' are not present")
    one_shots = one_shots[one_shots['One_Shot'] == 1]
    loops = loops[loops['One_Shot'] == 0]

'loop_id' and 'one_shot_intent' are not present


In [98]:
one_shot_cats = set(one_shots['category'])
loop_cats = set(one_shots['category'])

In [99]:
assert one_shot_cats == loop_cats
cats = {}
for cat in one_shot_cats:
    cats[cat] = {}
    cats[cat]["current_loop"] = len(loops[loops['category'] == cat])
    cats[cat]["current_one_shot"] = len(one_shots[one_shots['category'] == cat])
    cats[cat]["all"] = cats[cat]["current_loop"] + cats[cat]["current_one_shot"]
    cats[cat]['needed_each'] = math.floor((one_to_loop_ratio * cats[cat]["all"]) / (one_to_loop_ratio +1 + loop_unit_multi_sample_pct))


cats

{'Hi Hat': {'current_loop': 195,
  'current_one_shot': 1005,
  'all': 1200,
  'needed_each': 545},
 'Kick': {'current_loop': 195,
  'current_one_shot': 1005,
  'all': 1200,
  'needed_each': 545},
 'Clap': {'current_loop': 195,
  'current_one_shot': 1005,
  'all': 1200,
  'needed_each': 545}}

# next up  
made a way to import files. get their category and count by one shot or loop  

now i need to do the following:  
- figure out the number of samples to switch from one shot to loop in order to have a 50/50 split: done  
  - a small amount (ex: 20%) of loops should use more than 1 sample. need to take into account that we'll use a higher amount of samples for loops than oneshots to acheive a 50/50 split: done  
- identify one shot intent for each oneshot  
- doccument their intent
- all of this needs to work with the possibility that loops may or may not exist. can't overwrite existing data when making loops. loop ids can't clash with existing loop ids



# line to save for when 
keep this line for when i need to start generating new ids for the loops
loop_id_num = max(loops['loop_id'])

In [104]:
for cat in cats:
    if cats[cat]['current_one_shot'] > cats[cat]['needed_each']:
        remaining = cats[cat]['current_one_shot'] - cats[cat]['needed_each']
        tmp = one_shots.copy()
        tmp = tmp[tmp['category'] == cat]
        tmp = tmp.apply(lambda x: print(x))
        #print(tmp[tmp['category'] == cat].head(10))
        #print(f"remaining = {remaining}")
    elif cats[cat]['needed_each'] > cats[cat]['current_one_shot']:
        warnings.warn(f"not enough one shots to reach {one_to_loop_ratio}:1 ratio")

id
5050    [-0.00079345703, 0.0012817383, -0.0015563965, ...
5045    [0.0025024414, 0.008178711, 0.04171753, 0.0574...
5046    [0.01953125, 0.04446411, 0.007232666, -0.05889...
5053    [-0.013702393, -0.0057373047, 0.0054016113, 0....
5056    [-0.008087158, -0.04876709, -0.108947754, -0.0...
                              ...                        
1684    [-0.057907104, -0.011657715, -0.067840576, 0.0...
5159    [1.1920929e-06, 3.4987926e-05, -0.007843614, -...
5173    [0.0, 0.42367315, 0.42367315, 0.22498238, 0.22...
5151    [0.0, -0.10589975, -0.19219172, 0.11683482, 0....
5161    [0.0, -0.013096809, -0.008170664, 0.024011195,...
Name: waveform, Length: 1005, dtype: object
id
5050    44100
5045    44100
5046    44100
5053    44100
5056    44100
        ...  
1684    44100
5159    44100
5173    44100
5151    44100
5161    44100
Name: sample_rate, Length: 1005, dtype: object
id
5050                    808 samples/hi hat/hi hat (1).wav
5045                   808 samples/hi hat/hi hat (